<font size="10">Part 1</font>

In [1]:
import numpy as np
import cvxpy as cp
from environment import Environment_1 as ENV  # class object, see the file "environment.py"

<font size="6">Part 1.1</font>

1. Solving the system of Bellman equations explicitly

In [2]:
def Bellman_equa(gamma, policy, env):
    """ 
    Bellman equations
    Input:
        gamma (float) - reward discount factor
        policy (2d array) - policy, the sum of each row is 1
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        VF (1d array) - Calculated value function
    """
    
    # Initialize the system of linear equations
    # Calculate the value function by (A * VF = b)
    A = np.zeros([env.n_state, env.n_state])
    b = np.zeros(env.n_state)

    for s in range(env.n_state):  # sweep all the states in the state space
        A[s,s] = 1  # VF[s] = sum(policy[s,a] * p * (r + gamma * VF[s_])), hence the coefficient for VF[s] is 1
        for a in range(env.n_action):  # sweep all the actions in the action space
            for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                A[s,s_] -= policy[s,a] * p * gamma  # put the coefficients of VF[s_] to the left of the equation, hence minus
                b[s] += policy[s,a] * p * r  # keep the constants to the right of the equation

    # VF = A' * b, value function equals the procuct of inverse A and b
    if np.linalg.det(A) == 0: raise ValueError("A is not invertible !!!!!!!!!!")
    A_ = np.linalg.inv(A)
    VF = np.dot(A_, b)
    return VF

2. Iterative policy evaluation

In [3]:
def Iter_policy_eval(theta, gamma, policy, env):
    """ 
    Iterative Policy Evaluation
    Input:
        theta (float) - a small threshold > 0 determining accuracy of estimation
        gamma (float) - reward discount factor
        policy (2d array) - policy, the sum of each row is 1
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        VF (1d array) - Estimated value function
    """
    
    VF = np.zeros(env.n_state)  # initialize value function

    while 1:  # loop untill converge
        delta = 0  # initialize delta

        for s in range(env.n_state):  # sweep all the states in the state space
            v = VF[s]  # store the old value function

            # estimate the new value function
            temp = 0
            for a in range(env.n_action):  # sweep all the actions in the action space
                for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                    temp += policy[s,a] * p * (r + gamma * VF[s_])
            VF[s] = temp  # new value function
            
            delta = max(delta, abs(v - VF[s]))  # update the gap between the old and new value functions

        if delta < theta:  # stop when the gap is sufficiently small, i.e., the value function converges
            return VF

3. Value iteration

In [4]:
def Value_iter(theta, gamma, env):
    """ 
    Value iteration
    Input:
        theta (float) - a small threshold > 0 determining accuracy of estimation
        gamma (float) - reward discount factor
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        VF (1d array) - Estimated value function
    """

    VF = np.zeros(env.n_state)  # initialize value function

    while 1:  # loop untill converge
        delta = 0  # initialize delta

        for s in range(env.n_state):  # sweep all the states in the state space
            v = VF[s]  # store the old value function

            # estimate the new value function
            temp1 = []
            for a in range(env.n_action):  # sweep all the actions in the action space
                temp2 = 0
                for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                    temp2 += p * (r + gamma * VF[s_])
                temp1.append(temp2)
            VF[s] = max(temp1)  # new value function
            
            delta = max(delta, abs(v - VF[s]))  # update the gap between the old and new value functions

        if delta < theta:  # stop when the gap is sufficiently small, i.e., the value function converges
            return VF

4. Results

In [5]:
# Initialize the environment, set up parameters
the = 1e-10
gam = 0.95
Env = ENV()
pol = np.ones([Env.n_state, Env.n_action]) / Env.n_action  # initialize policy, the sum of each row is 1

# Calculate the value function via the three approaches
V_pi_1 = Bellman_equa(gam, pol, Env)
V_pi_2 = Iter_policy_eval(the, gam, pol, Env)
V_pi_3 = Value_iter(the, gam, Env)

# Print the results
print('Bellman equation:')
print(V_pi_1.reshape([5, -1]), '\n')
print('Iterative Policy Evaluation:')
print(V_pi_2.reshape([5, -1]), '\n')
print('Value iteration:')
print(V_pi_3.reshape([5, -1]), '\n')

Bellman equation:
[[ 2.17100208  4.7336156   2.07028049  1.26529444  1.77912239]
 [ 1.1180732   1.7821227   1.17409573  0.739174    0.56246548]
 [ 0.16279444  0.47788999  0.35198379  0.11045592 -0.18617038]
 [-0.54699155 -0.28473257 -0.28040463 -0.43990985 -0.7443105 ]
 [-1.10787684 -0.84936779 -0.80799244 -0.93799278 -1.23723244]] 

Iterative Policy Evaluation:
[[ 2.17100208  4.7336156   2.07028049  1.26529444  1.77912239]
 [ 1.1180732   1.7821227   1.17409573  0.739174    0.56246548]
 [ 0.16279445  0.47788999  0.35198379  0.11045592 -0.18617037]
 [-0.54699155 -0.28473257 -0.28040463 -0.43990985 -0.7443105 ]
 [-1.10787684 -0.84936779 -0.80799244 -0.93799278 -1.23723244]] 

Value iteration:
[[20.99734632 22.10246981 20.99734632 19.94747901 18.38284993]
 [19.94747901 20.99734632 19.94747901 18.95010506 18.0025998 ]
 [18.95010506 19.94747901 18.95010506 18.0025998  17.10246981]
 [18.0025998  18.95010506 18.0025998  17.10246981 16.24734632]
 [17.10246981 18.0025998  17.10246981 16.2473463

<font size="6">Part 1.2</font>

0. Relavent functions

In [6]:
def find_policy(gamma, V, env):
    """ 
    Find the optimal policy under the current value function
    Input:
        gamma (float) - reward discount factor
        V (1d array) - Input value function
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        pol (2d array) - Calculated optimum policy under the input value function
                         the first dimension is for states, the second dimension is for possible multiple choices
    """

    pol = [[0] for _ in range(env.n_state)]  # initialize a deterministic policy

    for s in range(env.n_state):  # sweep all the states in the state space
        temp1 = []
        for a in range(env.n_action):  # sweep all the actions in the action space
            temp2 = 0
            for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                temp2 += p * (r + gamma * V[s_])
            temp1.append(temp2)
                
        pol[s] = (np.unique(np.argwhere(temp1 == np.max(temp1)))).tolist()  # find the optimal policy under the current value function
        
    return pol

def print_policy(policy, env):
    """ 
    Print the policy for visualization
    Input:
        policy (1d array) - a deterministic policy
        env (class) - gridworld environment with the following parameters:
            env.n_state (int) - number of states
            env.action_text (1d array) - contain the unicode of the arrows for visualization
                                             left ←    down ↓    right →   up ↑
                                          ['\u2190', '\u2193', '\u2192', '\u2191']
    Output:
        policy_visual (1d array) - visualized policy, with arrows pointing to the moving direction
    """
    policy_visual = ['' for _ in range(env.n_state)]

    for s in range(env.n_state):
        lenth = len(policy[s])
        if lenth == 4:
            policy_visual[s] += 'o'  # 'o' means 4 directions are all available
        else:
            for a in range(lenth):
                policy_visual[s] += env.action_text[policy[s][a]]

    return policy_visual

1. Explicitly solving the Bellman optimality equation

In [7]:
def Bellman_opti_equa(gamma, env):
    """ 
    Bellman optimality equation
    Input:
        gamma (float) - reward discount factor
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        V_opt (1d array) - Calculated value function
        policy_opt (2d array) - Calculated optimum policy, the first dimension is for states, the second dimension is for possible multiple choices
    """
    
    ################# Bellman optimality equation #################
    VF = cp.Variable(env.n_state)  # initialize the optimization problem for cvx

    con = []  # initialize the constraints
    for s in range(env.n_state):  # sweep all the states in the state space
        bellman = []
        for a in range(env.n_action):  # sweep all the actions in the action space
            temp = 0
            for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                temp += p * (r + gamma * VF[s_])
            bellman.append(temp)  # right hand side of the Bellman optimality equation for each action

        # for state s, the constraints is to find the maximum of the right hand side of the Bellman optimality equation
        # '>=' cannot be set as '==', which will make the problem does not follow DCP rules.
        con.append(VF[s] >= cp.max(cp.hstack(bellman)))

    # The objective is to minimize the value function
    # if do not set this, the result will be very large (more than 100), since this problem has multiple solutions
    obj = cp.Minimize(cp.sum(VF)/env.n_state)  # divided by the number of states, weight each value in the value function equally

    # Solve this convex problem, find the optimal value function
    problem = cp.Problem(obj, con)
    problem.solve()
    V_opt = VF.value

    ################# Get Optimal Policy #################
    policy_opt = find_policy(gamma, V_opt, env)

    return V_opt, policy_opt

2. Policy iteration with iterative policy evaluation

In [8]:
def Policy_iter_with_Iter_policy_eval(theta, gamma, env):
    """ 
    Policy iteration with iterative policy evaluation
    Input:
        theta (float) - a small threshold > 0 determining accuracy of estimation
        gamma (float) - reward discount factor
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        VF (1d array) - Estimated value function
        policy (2d array) - Calculated optimum policy, the first dimension is for states, the second dimension is for possible multiple choices
    """
    
    VF = np.zeros(env.n_state)  # initialize value function
    policy = [[0] for _ in range(env.n_state)]  # initialize a deterministic policy

    while 1:  # loop untill policy converges
        ################# Policy Evaluation #################
        while 1:  # loop untill value function converges
            delta = 0  # initialize delta

            for s in range(env.n_state):  # sweep all the states in the state space
                v = VF[s]  # store the old value function

                # estimate the new value function
                temp = 0
                for p, s_, r in env.model[s][policy[s][0]]:  # sweep all the possibilities in model
                    temp += p * (r + gamma * VF[s_])
                VF[s] = temp  # new value function
            
                delta = max(delta, abs(v - VF[s]))  # update the gap between the old and new value functions

            if delta < theta:  # stop when the gap is sufficiently small, i.e., the value function converges
                break
        
        ################# Policy Improvement #################
        policy_old = policy.copy()  # store the old policy
        policy = find_policy(gamma, VF, env)  # find the new policy

        if policy_old == policy:  # stop when the old policy equals the new policy, i.e., the policy converges
            return VF, policy

3. Policy improvement with value iteration

In [9]:
def Policy_improvement_with_Value_iter(theta, gamma, env):
    """ 
    Policy improvement with Value iteration
    Input:
        theta (float) - a small threshold > 0 determining accuracy of estimation
        gamma (float) - reward discount factor
        env (class) - gridworld environment with the following parameters:
            env.n_state (int)    - number of states
            env.n_action (int)   - number of actions
            env.model (4d array) - (n_state) by (n_action) by (n) by (3) array, for state s and action a,
                                   there are n possibilities, each row is composed of (p, s_, r)
                                    p  - transition probability from (s,a) to (s_)   ### sum of the n possibilities -- n p equals 1
                                    s_ - next state
                                    r  - reward of the transition from (s,a) to (s_)
    Output:
        VF (1d array) - Estimated value function
        policy (2d array) - Calculated optimum policy, the first dimension is for states, the second dimension is for possible multiple choices
    """

    VF = np.zeros(env.n_state)  # initialize value function
    policy = [[0] for _ in range(env.n_state)]  # initialize a deterministic policy

    while 1:  # loop untill policy converges
        ################# Value iteration #################
        while 1:  # loop untill value function converges
            delta = 0  # initialize delta

            for s in range(env.n_state):  # sweep all the states in the state space
                v = VF[s]  # store the old value function

                # estimate the new value function
                temp1 = []
                for a in range(env.n_action):  # sweep all the actions in the action space
                    temp2 = 0
                    for p, s_, r in env.model[s][a]:  # sweep all the possibilities in model
                        temp2 += p * (r + gamma * VF[s_])
                    temp1.append(temp2)
                VF[s] = max(temp1)  # new value function
            
                delta = max(delta, abs(v - VF[s]))  # update the gap between the old and new value functions

            if delta < theta:  # stop when the gap is sufficiently small, i.e., the value function converges
                break
        
        ################# Policy Improvement #################
        policy_old = policy.copy()  # store the old policy
        policy = find_policy(gamma, VF, env)  # find the new policy

        if policy_old == policy:  # stop when the old policy equals the new policy, i.e., the policy converges
            return VF, policy

4. Results

In [10]:
# Initialize the environment, set up parameters
the = 1e-10
gam = 0.95
Env = ENV()

# Calculate the value function and find the optimal policy via the three approaches
V_opt_1, pol_opt_1 = Bellman_opti_equa(gam, Env)
V_opt_2, pol_opt_2 = Policy_iter_with_Iter_policy_eval(the, gam, Env)
V_opt_3, pol_opt_3 = Policy_improvement_with_Value_iter(the, gam, Env)

# Print the results
print('Bellman optimality equation:')
print(V_opt_1.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_1, Env))).reshape([5, -1]), '\n')

print('Policy iteration with iterative policy evaluation:')
print(V_opt_2.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_2, Env))).reshape([5, -1]), '\n')

print('Policy improvement with value iteration:')
print(V_opt_3.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_3, Env))).reshape([5, -1]), '\n')

Bellman optimality equation:
[[20.99734632 22.1024698  20.99734631 19.947479   18.38284995]
 [19.947479   20.99734631 19.947479   18.95010505 18.0025998 ]
 [18.95010506 19.947479   18.95010505 18.0025998  17.10246982]
 [18.00259981 18.95010505 18.00259979 17.10246981 16.24734633]
 [17.10246983 18.0025998  17.10246981 16.24734633 15.43497902]] 

[['→' 'o' '←' '←' 'o']
 ['↑' '↑' '↑' '↑' '←']
 ['↑' '↑' '↑' '↑' '↑']
 ['↑' '↑' '↑' '↑' '↑']
 ['↑' '↑' '←' '↑' '↑']] 

Policy iteration with iterative policy evaluation:
[[20.99734632 22.10246981 20.99734632 19.94747901 18.38284993]
 [19.94747901 20.99734632 19.94747901 18.95010506 18.0025998 ]
 [18.95010506 19.94747901 18.95010506 18.0025998  17.10246981]
 [18.0025998  18.95010506 18.0025998  17.10246981 16.24734632]
 [17.10246981 18.0025998  17.10246981 16.24734632 15.43497901]] 

[['→' 'o' '←' '←' 'o']
 ['→' '↑' '←↑' '←↑' '←']
 ['→' '↑' '←↑' '←↑' '←↑']
 ['→' '↑' '←↑' '←↑' '←↑']
 ['→' '↑' '←↑' '←↑' '←↑']] 

Policy improvement with value iterati

5. Problem

In [11]:
# due to estimation problem, they are not equal
print(V_opt_2[0])
print(V_opt_2[6])

20.997346322099883
20.99734632212662


In [12]:
# Therefore, round the value function first, and then find the policy

# Round the value function
V_opt_1_ = np.round(V_opt_1,3)
V_opt_2_ = np.round(V_opt_2,3)
V_opt_3_ = np.round(V_opt_3,3)

# Find the policy again
pol_opt_1_ = find_policy(gam, V_opt_1_, Env)
pol_opt_2_ = find_policy(gam, V_opt_2_, Env)
pol_opt_3_ = find_policy(gam, V_opt_3_, Env)

# Print the results
print('Bellman optimality equation:')
print(V_opt_1_.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_1_, Env))).reshape([5, -1]), '\n')

print('Policy iteration with iterative policy evaluation:')
print(V_opt_2_.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_2_, Env))).reshape([5, -1]), '\n')

print('Policy improvement with value iteration:')
print(V_opt_3_.reshape([5, -1]), '\n')
print((np.array(print_policy(pol_opt_3_, Env))).reshape([5, -1]), '\n')

Bellman optimality equation:
[[20.997 22.102 20.997 19.947 18.383]
 [19.947 20.997 19.947 18.95  18.003]
 [18.95  19.947 18.95  18.003 17.102]
 [18.003 18.95  18.003 17.102 16.247]
 [17.102 18.003 17.102 16.247 15.435]] 

[['→' 'o' '←' '←' 'o']
 ['→↑' '↑' '←↑' '←↑' '←']
 ['→↑' '↑' '←↑' '←↑' '←↑']
 ['→↑' '↑' '←↑' '←↑' '←↑']
 ['→↑' '↑' '←↑' '←↑' '←↑']] 

Policy iteration with iterative policy evaluation:
[[20.997 22.102 20.997 19.947 18.383]
 [19.947 20.997 19.947 18.95  18.003]
 [18.95  19.947 18.95  18.003 17.102]
 [18.003 18.95  18.003 17.102 16.247]
 [17.102 18.003 17.102 16.247 15.435]] 

[['→' 'o' '←' '←' 'o']
 ['→↑' '↑' '←↑' '←↑' '←']
 ['→↑' '↑' '←↑' '←↑' '←↑']
 ['→↑' '↑' '←↑' '←↑' '←↑']
 ['→↑' '↑' '←↑' '←↑' '←↑']] 

Policy improvement with value iteration:
[[20.997 22.102 20.997 19.947 18.383]
 [19.947 20.997 19.947 18.95  18.003]
 [18.95  19.947 18.95  18.003 17.102]
 [18.003 18.95  18.003 17.102 16.247]
 [17.102 18.003 17.102 16.247 15.435]] 

[['→' 'o' '←' '←' 'o']
 ['→↑' '↑' 